# SQL 的五十道練習

> 函數

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import sqlite3
import unittest
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 從 NBA 球員資料表 `players` 依據身高 `heightMeters` 與 `weightKilograms` 衍生計算欄位 `bmi`，使用 `ROUND()` 將 `bmi` 的小數點位數調整為 2 位。

\begin{equation}
BMI = \frac{weight_{kg}}{height_{m}^2}
\end{equation}

- 預期輸入：SQL 查詢語法。
- 預期輸出：(495, 3) 的查詢結果。

```
     heightMeters  weightKilograms    bmi
0            2.06            113.4  26.72
1            2.01            108.0  26.73
2            2.03            106.6  25.87
3            2.08            120.2  27.78
4            1.98             97.5  24.87
..            ...              ...    ...
490          2.01            104.3  25.82
491          2.08            106.1  24.52
492          1.78             88.5  27.93
493          1.98             90.7  23.14
494          1.96             83.9  21.84

[495 rows x 3 columns]
```

In [2]:
calculate_rounded_bmi =\
"""
-- SQL 查詢語法起點
SELECT heightMeters,
       weightKilograms,
       ROUND(weightKilograms / (heightMeters*heightMeters), 2) AS bmi
  FROM players;
-- SQL 查詢語法終點
"""

## 從 NBA 球員生涯攻守資料表 `career_summaries` 中選擇 `assists` 與 `turnovers` 衍生計算助攻失誤比 `ast_to_ratio`，使用 `CAST()` 讓 `ast_to_ratio` 的資料類型為浮點數 `REAL`。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(495, 3) 的查詢結果。

```
    assists turnovers  ast_to_ratio
0        16        23      0.695652
1        65        28      2.321429
2       660       793      0.832282
3       746       446      1.672646
4      2020      1587      1.272842
..      ...       ...           ...
490     871       290      3.003448
491    1666      1422      1.171589
492    1368       670      2.041791
493     600       457      1.312910
494     223       206      1.082524

[495 rows x 3 columns]
```

In [3]:
calculate_ast_to_ratio =\
"""
-- SQL 查詢語法起點
SELECT assists,
       turnovers,
       CAST(assists AS REAL) / turnovers AS ast_to_ratio
  FROM career_summaries;
-- SQL 查詢語法終點
"""

## 從 NBA 球員資料表 `players` 中選擇 `firstName` 與 `lastName`，使用 `UPPER()` 以及 `LOWER()` 將 `firstName` 調整為全小寫、將 `lastName` 調整為全大寫。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(495, 2) 的查詢結果。

```
    lower_first_name upper_last_name
0             lebron           JAMES
1            carmelo         ANTHONY
2             udonis          HASLEM
3             dwight          HOWARD
4              andre        IGUODALA
..               ...             ...
490          anthony            GILL
491             will          MAGNAY
492          facundo        CAMPAZZO
493             nate         DARLING
494          brodric          THOMAS

[495 rows x 2 columns]
```

In [4]:
select_lower_firstname_upper_lastname =\
"""
-- SQL 查詢語法起點
SELECT LOWER(firstName) AS lower_first_name,
       UPPER(lastName) AS upper_last_name
  FROM players;
-- SQL 查詢語法終點
"""

## 從 2020 台灣總統大選資料表 `presidential` 中利用聚合函數彙總有多少人投下總統大選的選票 `total_votes`（有效票數），透過連結 `county`、`town` 與 `village` 計算台灣共有幾個獨一選舉區 `n_electoral_area`。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(1, 2) 的查詢結果。

```
   total_votes  n_electoral_area
0     14300940              7737
```

In [5]:
summarize_presidential_votes_electoral_area =\
"""
-- SQL 查詢語法起點
SELECT SUM(votes) AS total_votes,
       COUNT(DISTINCT county || town || village) AS n_electoral_area
  FROM presidential;
-- SQL 查詢語法終點
"""

## 從新冠肺炎每日報告資料表 `daily_report` 中利用聚合函數彙總截至 2021-01-31 全世界有出現確診數的國家有幾個 `n_countries_affected`、全世界總確診數 `total_confirmed`、全世界總痊癒數 `total_recovered` 以及全世界總死亡數 `total_deaths`。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(1, 4) 的查詢結果。

```
   n_countries_affected  total_confirmed  total_recovered  total_deaths
0                   192        102965869         57049348       2227910
```

In [6]:
summarize_daily_report_stats =\
"""
-- SQL 查詢語法起點
SELECT COUNT(DISTINCT Country_Region) AS n_countries_affected,
       SUM(Confirmed) AS total_confirmed,
       SUM(Recovered) AS total_recovered,
       SUM(Deaths) AS total_deaths
  FROM daily_report;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All.

In [7]:
class TestFunctions(unittest.TestCase):
    def test_calculate_rounded_bmi(self):
        rounded_bmi = pd.read_sql(calculate_rounded_bmi, conn)
        self.assertEqual(rounded_bmi.shape, (495, 3))
        np.testing.assert_equal(rounded_bmi.columns.values,
                               np.array(['heightMeters', 'weightKilograms', 'bmi']))
        np.testing.assert_almost_equal(rounded_bmi['bmi'].values[:5],
                               np.array([26.72, 26.73, 25.87, 27.78, 24.87]))
        np.testing.assert_almost_equal(rounded_bmi['bmi'].values[-5:],
                               np.array([25.82, 24.52, 27.93, 23.14, 21.84]))

    def test_calculate_ast_to_ratio(self):
        ast_to_ratio = pd.read_sql(calculate_ast_to_ratio, conn)
        self.assertEqual(ast_to_ratio.shape, (495, 3))
        np.testing.assert_equal(ast_to_ratio.columns.values,
                               np.array(['assists', 'turnovers', 'ast_to_ratio']))
        self.assertEqual(str(pd.read_sql(calculate_ast_to_ratio, conn)['ast_to_ratio'].dtype), 'float64')
        
    def test_select_lower_firstname_upper_lastname(self):
        lower_firstname_upper_lastname = pd.read_sql(select_lower_firstname_upper_lastname, conn)
        self.assertEqual(lower_firstname_upper_lastname.shape, (495, 2))
        np.testing.assert_equal(lower_firstname_upper_lastname['lower_first_name'].values[:5],
                               np.array(['lebron', 'carmelo', 'udonis', 'dwight', 'andre']))
        np.testing.assert_equal(lower_firstname_upper_lastname['upper_last_name'].values[:5],
                               np.array(['JAMES', 'ANTHONY', 'HASLEM', 'HOWARD', 'IGUODALA']))

    def test_summarize_daily_report_stats(self):
        daily_report_stats = pd.read_sql(summarize_daily_report_stats, conn)
        self.assertEqual(daily_report_stats.shape, (1, 4))
        np.testing.assert_equal(daily_report_stats.loc[0, :].values,
                               np.array([192, 102965869, 57049348, 2227910]))

    def test_summarize_presidential_votes_electoral_area(self):
        presidential_votes_electoral_area = pd.read_sql(summarize_presidential_votes_electoral_area, conn)
        self.assertEqual(presidential_votes_electoral_area.shape, (1, 2))
        np.testing.assert_equal(presidential_votes_electoral_area.loc[0, :].values,
                               np.array([14300940, 7737]))

suite = unittest.TestLoader().loadTestsFromTestCase(TestFunctions)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_calculate_ast_to_ratio (__main__.TestFunctions) ... ok
test_calculate_rounded_bmi (__main__.TestFunctions) ... ok
test_select_lower_firstname_upper_lastname (__main__.TestFunctions) ... ok
test_summarize_daily_report_stats (__main__.TestFunctions) ... ok
test_summarize_presidential_votes_electoral_area (__main__.TestFunctions) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.194s

OK


In [8]:
print("您在 {} 道 SQL 練習中答對了 {} 題。".format(number_of_test_runs, number_of_successes))

您在 5 道 SQL 練習中答對了 5 題。
